# Feature Engineering (Scooter & Locations)

## Preparations

modules

In [1]:
import pandas as pd  # working with dataframes
from helper_functions import (transform_to_timestamp, isNaN, 
                              get_timedelta, calc_distance, 
                              zone_change, inside_timeframe)  # own functions

options

In [2]:
# automatic reloading of modules and functions before each cell
%load_ext autoreload
%autoreload 2

loading data

In [3]:
df_sorted = pd.read_csv('data/df_sorted.csv')
df_sorted['time'] = df_sorted.time.map(transform_to_timestamp)
df_scooter_level = pd.read_csv('data/scooter_level.csv', index_col=0)
locations_3dp = pd.read_csv('data/locations_3dp.csv')

## Feature Engineering

### General

#### parking time (scooters are not being used)

In [4]:
df_sorted_shifted = df_sorted.shift(-1)
df_sorted_shifted.columns = [x + '_shift' for x in list(df_sorted.columns)]

In [5]:
df_sorted.shape == df_sorted_shifted.shape

True

In [6]:
df_features = pd.concat([df_sorted, df_sorted_shifted], axis=1)

create indicator if the next observation is still the same scooter

In [7]:
df_features['same_scooter'] =  (isNaN(df_features.id_shift) | 
                                (df_features.id == df_features.id_shift))

In [8]:
df_features['parking'] = (((df_features.lat == df_features.lat_shift) & 
                           (df_features.lon == df_features.lon_shift)) |
                          (~ df_features.same_scooter) |
                          (isNaN(df_features.lat_shift))).astype(int)

We can now calculate the seconds between timestamps and set it to 0 if a scooter was in use to create the total time parking.

In [9]:
df_features['time_parked'] = [get_timedelta(x) for x in df_features[['time_shift', 'time']].values]
df_features.loc[df_features.parking==0 | ~df_features.same_scooter, 'time_parked'] = 0

#### distance travelled (proxy via haversine)

In [10]:
df_features['distance'] = [calc_distance(x) for x in df_features[['lat', 'lon', 'lat_shift', 'lon_shift']].values]
df_features.loc[~df_features.same_scooter, 'distance'] = 0

In [11]:
df_features.head(20)

,id,lat,lon,time,id_shift,lat_shift,lon_shift,time_shift,same_scooter,parking,time_parked,distance
0,ABKIFV,48.186981,16.413081,05:28:11,ABKIFV,48.186981,16.413081,05:29:13,True,1,62.0,0.000000
1,ABKIFV,48.186981,16.413081,05:29:13,ABKIFV,48.186981,16.413081,05:30:16,True,1,63.0,0.000000
2,ABKIFV,48.186981,16.413081,05:30:16,ABKIFV,48.186981,16.413081,05:31:19,True,1,63.0,0.000000
3,ABKIFV,48.186981,16.413081,05:31:19,ABKIFV,48.186981,16.413081,05:32:22,True,1,63.0,0.000000
4,ABKIFV,48.186981,16.413081,05:32:22,ABKIFV,48.186981,16.413081,05:33:24,True,1,62.0,0.000000
5,ABKIFV,48.186981,16.413081,05:33:24,ABKIFV,48.186981,16.413081,05:34:27,True,1,63.0,0.000000
6,ABKIFV,48.186981,16.413081,05:34:27,ABKIFV,48.186981,16.413081,05:35:30,True,1,63.0,0.000000
7,ABKIFV,48.186981,16.413081,05:35:30,ABKIFV,48.186981,16.413081,05:36:33,True,1,63.0,0.000000
8,ABKIFV,48.186981,16.413081,05:36:33,ABKIFV,48.186981,16.413081,05:37:35,True,1,62.0,0.000000
9,ABKIFV,48.186981,16.413081,05:37:35,ABKIFV,48.209058,16.370608,05:38:38,True,0,0.0,3992.029107


We can clearly differ times of usage to times of parking and also see quite some outliers in the distance travelled. For example travelling almost four kilometers within a minute corresponds to 240 km/h and is very unlikely (even with a car, i.e. relocating the scooter) inside a city. Without further domain knowledge we must ignore these outlierst for now.

In [12]:
df_features[df_features.distance>0].distance.describe()

count    57243.000000
mean        61.876936
std        272.859576
min          0.024710
25%          3.763409
50%          7.958448
75%         19.146028
max       8077.506133
Name: distance, dtype: float64

On average if a scooter moves, it travels ~ eight meters per minute, which sounds legit.

### Create features on scooter level

Next step is to create valuable features out of these two new insights on a scooter level. Reminder, this is what we have so far:

In [13]:
df_scooter_level.head()

,n_seen,unique_locs
FPWXUQ,911,93
NLMWIG,911,46
ALUIQD,911,21
RNUSIE,911,40
JHQBFL,911,59


We aggregate on scooter level for the standard deviation and the total sum of seconds parked and meters travelled.

In [14]:
df_features_scooter = df_features.groupby('id')[['time_parked', 'distance']].agg(['std', 'sum'])

In [15]:
df_features_scooter.columns = ['time_parked_std', 'time_parked_sum', 'distance_std', 'distance_sum']

We collect the number of unique three decimal places locations a scooter was seen in.

In [16]:
df_n_locations = df_sorted[['id', 'lon', 'lat']].set_index('id').apply(lambda x: round(x, 3))\
                                                .drop_duplicates().groupby('id').size()\
                                                .to_frame(name='unique_3dp_locs')

And join it all together.

In [17]:
df_features_scooter = df_features_scooter.merge(df_scooter_level,
                                                how='left',
                                                left_index=True, 
                                                right_index=True)\
                                         .merge(df_n_locations,
                                                how='left',
                                                left_index=True, 
                                                right_index=True).fillna(0)

In [18]:
df_features_scooter.head()

,time_parked_std,time_parked_sum,distance_std,distance_sum,n_seen,unique_locs,unique_3dp_locs
id,,,,,,,
ABKIFV,16.675280,47990.0,169.310141,7722.699262,816,60,7.0
ACODMP,23.729405,46756.0,63.685439,11332.193648,881,146,47.0
ADIBPW,12.012686,55643.0,2.048582,301.460516,909,33,2.0
ADKPIH,15.772084,42385.0,44.426778,1685.282999,712,46,4.0
ADULSM,13.802959,54515.0,31.007958,1626.687718,902,44,4.0


This should be enough for an MVP of classifying a scooter's activity/demand.

### Create features on location level

Again, this is what we have so far.

In [19]:
locations_3dp.head()

,lat,lon,scooter_count,morning_count,evening_count
0,48.185,16.416,2634,61.0,3.0
1,48.187,16.413,746,18.0,1.0
2,48.186,16.416,478,17.0,0.0
3,48.226,16.361,3234,13.0,5.0
4,48.200,16.365,1993,11.0,1.0


Round the locations to 3 digits.

In [20]:
df_features_location = df_features
for col in ['lat', 'lon', 'lat_shift', 'lon_shift']:
    df_features_location[col] = round(df_features[col], 3)

Create indicators if a scooter left or entered the zone.

In [21]:
df_features_location['rent_start'] = ((df_features_location.distance > 0) & 
                                      (df_features_location.distance.shift(1)==0) &
                                      (df_features.same_scooter)).astype(int)

In [22]:
df_features_location['zone_left'] = [zone_change(x) for x in 
                                       df_features_location[['lat', 'lon', 'lat_shift', 'lon_shift']].values]
df_features_location.loc[~df_features_location.same_scooter, 'zone_left'] = 0

In [23]:
df_features_location['zone_entered'] = df_features_location.zone_left.shift(1).fillna(0).astype(int)

Create indicators for rental starts during peak times.

In [24]:
early_peak = ('5:00:00', '8:29:00')
late_peak = ('19:30:00', '20:29:00')

In [25]:
df_features_location['early_peak'] = [inside_timeframe(x, early_peak[0], early_peak[1]) 
                                      for x in df_features['time'].values]
df_features_location['late_peak'] = [inside_timeframe(x, late_peak[0], late_peak[1]) 
                                      for x in df_features['time'].values]

Control for an actual rent start.

In [26]:
df_features_location['early_peak'] = df_features_location.early_peak*df_features_location.rent_start
df_features_location['late_peak'] = df_features_location.late_peak*df_features_location.rent_start

In [27]:
df_features_location.head()

,id,lat,lon,time,id_shift,lat_shift,lon_shift,time_shift,same_scooter,parking,time_parked,distance,rent_start,zone_left,zone_entered,early_peak,late_peak
0,ABKIFV,48.187,16.413,05:28:11,ABKIFV,48.187,16.413,05:29:13,True,1,62.0,0.0,0,0,0,0,0
1,ABKIFV,48.187,16.413,05:29:13,ABKIFV,48.187,16.413,05:30:16,True,1,63.0,0.0,0,0,0,0,0
2,ABKIFV,48.187,16.413,05:30:16,ABKIFV,48.187,16.413,05:31:19,True,1,63.0,0.0,0,0,0,0,0
3,ABKIFV,48.187,16.413,05:31:19,ABKIFV,48.187,16.413,05:32:22,True,1,63.0,0.0,0,0,0,0,0
4,ABKIFV,48.187,16.413,05:32:22,ABKIFV,48.187,16.413,05:33:24,True,1,62.0,0.0,0,0,0,0,0


Aggregate values to zone level and merge with location dataframe. First the indicators.

In [28]:
df_features_location_ind = df_features_location.groupby(['lat', 'lon'])[['zone_entered', 'zone_left',
                                                                         'rent_start',
                                                                         'early_peak', 'late_peak']]\
                                               .agg('sum').fillna(0)

Then the metrical ones, where we also want the standard deviation.

In [29]:
df_features_location_met = df_features_location.groupby(['lat', 'lon'])[['time_parked', 'distance']]\
                                               .agg(['std', 'sum']).fillna(0)
df_features_location_met.columns = ['time_parked_std', 'time_parked_sum', 
                                    'distance_std', 'distance_sum']

Now join together.

In [30]:
df_features_location = df_features_location_met.merge(df_features_location_ind, how='outer',
                                                      left_index=True, right_index=True)

In [31]:
df_features_location.sort_values('rent_start', ascending=False).head()

,,time_parked_std,time_parked_sum,distance_std,distance_sum,zone_entered,zone_left,rent_start,early_peak,late_peak
lat,lon,,,,,,,,,
48.185,16.416,25.649346,130711.0,549.008378,182582.614506,103,161,123,89,0
48.200,16.361,13.678259,341469.0,2.670900,2469.462988,8,5,105,13,17
48.194,16.362,12.855193,242094.0,27.337896,3883.813035,3,3,100,16,13
48.196,16.339,14.677699,275893.0,64.335619,15939.956328,49,51,99,31,5
48.199,16.367,15.797097,172589.0,32.148110,5093.076190,18,20,94,13,11


Define coordinates of location as indexes.

In [32]:
for col in ['lat', 'lon']:
    locations_3dp[col] = round(locations_3dp[col], 3) # redo rounding to have exact same values
locations_3dp.set_index(['lat', 'lon'], inplace=True)

In [33]:
df_features_location = locations_3dp.merge(df_features_location, how='outer', 
                                           left_index=True, right_index=True)

In [34]:
df_features_location.sort_values('morning_count', ascending=False).head()

,,scooter_count,morning_count,evening_count,time_parked_std,time_parked_sum,distance_std,distance_sum,zone_entered,zone_left,rent_start,early_peak,late_peak
lat,lon,,,,,,,,,,,,
48.185,16.416,2634,61.0,3.0,25.649346,130711.0,549.008378,182582.614506,103,161,123,89,0
48.187,16.413,746,18.0,1.0,23.181728,39271.0,536.650062,75663.369909,21,38,47,43,4
48.186,16.416,478,17.0,0.0,24.785012,24392.0,279.621734,15870.912956,37,54,28,23,0
48.226,16.361,3234,13.0,5.0,16.054756,191409.0,36.001789,8961.148129,17,25,71,30,6
48.200,16.365,1993,11.0,1.0,19.056158,113893.0,27.913234,4711.171794,2,12,47,9,2


This should be enough for an MVP of clustering a location's activity/demand.

## Saving results

Before we create our reallocation recommendation, we want to save our dataframes.

In [35]:
print(df_features_scooter.shape)
print(df_features_location.shape)

(997, 7)
(2050, 12)


In [36]:
df_features_scooter.to_csv('data/features_scooter.csv')
df_features_location.to_csv('data/features_location.csv')